In [1]:
# import the necessary packages
import os
import uuid
from PIL import Image
from IPython.display import clear_output

In [2]:
# Simple progressBar to monitor progress :D
# Credits: https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/

def updateProgressBar(progress, bar_length = 50):
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    # Clear current cell output 
    clear_output(wait = True)
    # Print progress!
    print("Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100))

In [4]:
# Set dataset path location
BASE_PATH = os.path.join(os.getcwd(), '..', 'dataset')
DATASET_FOLDER = os.path.join('mixed-cropped', 'ori')

# Set appropiate target image size
MAX_SIZE = 320

In [5]:
# Resize Image according to given files structure:
# PathToDir/ 
#     Label_A/ 
#         PathToImg_1.jpg
#         PathToImg_2.jpg
#         PathToImg_n.jpg
#         ....
#     Label_B/
#         PathToImg_1.jpg
#         PathToImg_2.jpg
#         PathToImg_n.jpg
#         ....
#     Label_n/
#         ....

# Generate Input and Output Location
IN_PATH = os.path.join(BASE_PATH, DATASET_FOLDER)
OUT_PATH = os.path.join(BASE_PATH, '{}-{}'.format(DATASET_FOLDER, str(MAX_SIZE)))
# Check if output folder doesn't exist, create new one
if os.path.exists(OUT_PATH) is False:
    os.mkdir(OUT_PATH)

# Open and Check input folder files number
labels = os.listdir(IN_PATH)
num_files = 0
cnt = 0

for label in labels:
    files = os.listdir(os.path.join(IN_PATH, label))
    # Calculate num files for progress
    num_files += len(files)
    # Check output label folder exists
    if os.path.exists(os.path.join(OUT_PATH, label)) is False:
        os.mkdir(os.path.join(OUT_PATH, label))

# Enumerate every label in IN_PATH
for label in labels:
    files = os.listdir(os.path.join(IN_PATH, label))
    # Enumerate every file in label folder
    for idx, file in enumerate(files):
        fName, fExt = os.path.splitext(file)
        if fExt is not '':
            # Generate spesicic file id, make sure everthing unique!
            fId = str(uuid.uuid4())[:14]
            # Load Image
            image = Image.open(os.path.join(IN_PATH, label, str(file))).convert('RGB')
            # Calculate presentage from actual to given size
            w, h = image.size
            if w > h:
                p = MAX_SIZE / w
            else:
                p = MAX_SIZE / h
            # Calculate target resized size that maintain original image ratio
            tw = int(w * p)
            th = int(h * p)
            # Resizing Image
            image = image.resize((tw, th))
            # Save Resized image
            # image.save(os.path.join(OUT_PATH, label, '{}-{}{}{}'.format(label, fId, idx, fExt)), 'JPEG')
            image.save(os.path.join(OUT_PATH, label, '{}{}'.format(fName, fExt)), 'JPEG')
            # Update Progress Bar
            cnt += 1
            updateProgressBar(cnt / num_files)

# Progress full/ complete task
updateProgressBar(1)

Progress: [##################################################] 100.0%


In [8]:
import splitfolders

BASE_PATH = os.path.join(os.getcwd(), '..', 'dataset')
DATASET_FOLDER = os.path.join('mixed-cropped', 'ori-320-sm')

splitfolders.ratio(os.path.join(BASE_PATH, DATASET_FOLDER), output = os.path.join(BASE_PATH, 'mixed-cropped'), seed = 1092, ratio = (0.5, 0.5, 0), group_prefix = None)